In [1]:
import tabula
from mbmutils import mu
import pandas as pd

In [2]:
df = tabula.read_pdf(f"{mu.get_full_path('data')}/gunsvsmotorvehicles22.pdf", pages='all')

Error from tabula-java:
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.





CalledProcessError: Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/bladnman/opt/anaconda3/lib/python3.9/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', '/Users/bladnman/code/notebooks/gun-data/data//gunsvsmotorvehicles22.pdf']' returned non-zero exit status 1.